In [26]:
import IPython

import ipykernel.ipkernel
from ipython_genutils.py3compat import builtin_mod, PY3, unicode_type, safe_unicode

from IPython.utils.tokenutil import token_at_cursor, line_at_cursor


try:
    import stringdisplays, markdown, tokens, phrases, magics
except:
    with __import__('importnb').Notebook(): from pidgin import stringdisplays, markdown, tokens, phrases, magics

import sys
import textwrap
import black
import mimetypes
import collections, html
import base64
import sys
import IPython
from traitlets import *
ip = self = IPython.get_ipython()
import trio
import ipykernel
import trio
import importlib
import traitlets
import jinja2
from tornado import gen
import asyncio
import tokenize
import itertools, operator
import toolz.curried as toolz
import tokenize
import io

In [27]:
def expression_tokens(str): return list(tokenize.generate_tokens(io.StringIO(str).readline))

In [28]:
s = "print; range; crap "

In [29]:
def split_expression(str, *expressions):
    """Split an expression on the semi colons."""
    start = 0
    for id in toolz.pipe(
        str, expression_tokens, 
        toolz.filter(toolz.compose(
            ';'.__eq__, operator.attrgetter('string')
        )), 
        toolz.map(toolz.compose(
            toolz.second,
            operator.attrgetter('start')
        )),
        list
    ) + [len(str)]:
        expressions += str[start:id],
        start = id + 1
    return expressions

In [32]:
import ipykernel

In [13]:
class PidginShell(ipykernel.zmqshell.ZMQInteractiveShell):
    markdown = Bool(True)
    template = Bool(True)
    expressions = Bool(True)
    tangle_expressions = Bool(True)
    
    def user_expressions(self, expressions): 
        if self.expressions:
            return trio.run(self.async_user_expressions, expressions)
        return {}

    async def async_user_expressions(self, expressions):
        async with trio.open_nursery() as nursery: 
            for key, expression in expressions.items():
                nursery.start_soon(self.single_user_expression, expression)
                
    async def single_user_expression(self, code):
        expressions = split_expression(code)
        results = super().user_expressions(dict(
            zip(expressions, map(self.input_transformer_manager.transform_cell, expressions))))
        
        IPython.display.display(IPython.display.Markdown('''`>>> {}`'''.format(code.strip('`'))))
        
        error_msg = []
        for expression, result in results.items():
            expressions_ns = '`{}`'.format(expression.strip())
            if result['status'] == 'error':
                error_msg.extend(result['traceback'])
            elif expressions_ns in self.user_ns:
                display = self.user_ns[expressions_ns]
                IPython.display.publish_display_data(
                    result['data'], update=True, transient={
                        'display_id': display.display_id}
                )
        
        if error_msg:
            IPython.display.publish_display_data({'text/plain': ''.join(error_msg)})
            
        if result['status'] == 'ok':
            display = self.user_ns[
                "`{}`".format(code.strip())
            ] = self.user_ns[expressions_ns] = IPython.display.DisplayHandle()
            IPython.display.publish_display_data(result['data'], transient={
                'display_id': display.display_id})
        if error_msg: result['status'] = 'error'
        return result

In [114]:
def requote(str, token='"""'):
    if token in str: token = "'''"
    return token+str+token

    ipykernel.ipkernel.IPythonKernel.do_execute??

In [115]:
class PidginKernel(ipykernel.ipkernel.IPythonKernel):
    shell_class = traitlets.Type(PidginShell)
    def __init__(self, **kwargs):
        super().__init__(**kwargs)
        stringdisplays.load_ipython_extension(self.shell)
        phrases.load_ipython_extension(self.shell)
        magics.load_ipython_extension(self.shell)
        self.shell.user_ns['__import__'] = importlib.import_module
        
    language_info = {**ipykernel.ipkernel.IPythonKernel.language_info, **{
        'name': 'pidgin',
        'display_name': 'pidgin',
        'nbconvert_exporter': 'python',
        'file_extension': '.md.ipynb',
        "argv": "python -m pidgin.kernel -f {connection_file}".split(),
    }}        
    
    @gen.coroutine 
    def do_execute(self, code, silent, store_history=True, user_expressions=None, allow_stdin=False):
        if user_expressions is None: user_expressions = {}
        
        if self.shell.markdown and not code.lstrip().startswith('%'): 
            source = markdown.renderer(code, user_expressions=user_expressions)
        else: source = code
        
        silent = silent or not source
        display = None
        
        if self.shell.markdown and source:
            if code.strip() and code.splitlines()[0].strip():
                display = IPython.display.display( IPython.display.Markdown(code), display_id=True)
            
        if not source:
            if self.shell.template:
                code = jinja2.Template(code).render(**self.shell.user_ns)
            result = super().do_execute(requote(
                code
            ), False, store_history=False, user_expressions={}, allow_stdin=False)
        else: 
            result = super().do_execute(source, silent, store_history=store_history, user_expressions={}, allow_stdin=allow_stdin)

        with self._cancel_on_sigint(asyncio.ensure_future(result)):
            result = yield result
            
        if user_expressions:
            IPython.display.display(IPython.display.Markdown('---'))
            self.shell.user_expressions(user_expressions)
            
        display and self.shell.template and display.update(
            IPython.display.Markdown(jinja2.Template(code).render(**self.shell.user_ns))
        )
        return result

In [116]:
def define_urls(str):
    while str[0] != 'h' and str[0] == str[1]: str = str[1:-1]
    for body in str.split('http')[1:]:
        setattr(__import__('__main__'), f"http{body.split()[0]}", f"http{body.split()[0]}" )

In [117]:
import jupyter_client.manager

In [118]:
manager = jupyter_client.kernelspec.KernelSpecManager()
import json

In [119]:
import IPython

In [120]:
IPython.core.interactiveshell.InteractiveShellABC.register(PidginShell)

__main__.PidginShell

    %%file kernel.py
    with __import__('importnb').Notebook():
        from .kernel import PidginKernel
        
    if __name__ == '__main__':
        from ipykernel.kernelapp import IPKernelApp
        IPKernelApp.launch_instance(kernel_class=PidginKernel)
    

In [14]:
if __name__ == '__main__':

    with open('C:/Users/deathbeds/pidgin/pidgin/kernel.json', 'w') as f:
        json.dump(PidginKernel.language_info, f)
    manager.install_kernel_spec('C:/Users/deathbeds/pidgin/pidgin/')
    manager.get_all_specs()        